# Task 2: Recommendation Engine - Skeleton Notebook

This notebook provides a very basic example for the notebook you are expected to submit for Task 2 of the Final Project. The main purpose is that we can try different examples to get a better sense of your approach. Compared to Task 1 (Kaggle Competition), we don't have any objective means to evaluate the recommendations. 

Some general comments:
* You can import any data you need. This particularly includes your cleaned version of the Used Cars dataset; there's no need to show the data cleaning / preprocessing steps in this notebook.
* You can also import your code in form of external Python (.py) script. You're actually encouraged to do so to keep this notebook light and uncluttered.
* Please consider this notebook as an example and not to set specific requirements. As long there is a section where we can easily test your solution, it should be fine.

## Setting up the Notebook

In [6]:
import pandas as pd
import numpy as np
import heapq
import utils

## Load the Data

For this example, we use a simplified version of the dataset with only 100 data sample, each with only 6 features

In [114]:
df_sample = pd.read_csv('../data/train.csv')
df_original = pd.read_csv('../data/train.csv')

df_sample.head()

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,...,mileage,omv,arf,opc_scheme,lifespan,eco_category,features,accessories,indicative_price,price
0,1030324,BMW 3 Series 320i Gran Turismo M-Sport,bmw,320i,1 owner! 320i gt m-sports model! big brake kit...,2013.0,NaN,09-dec-2013,luxury sedan,"parf car, premium ad car, low mileage car",...,73000.0,45330.0,50462.0,NaN,NaN,uncategorized,"5 doors gt, powerful and fuel efficient 2.0l t...","bmw i-drive, navigation, bluetooth/aux/usb inp...",NaN,71300.0
1,1021510,Toyota Hiace 3.0M,NaN,hiace,high loan available! low mileage unit. wear an...,2014.0,NaN,26-jan-2015,van,premium ad car,...,110112.0,27502.0,1376.0,NaN,25-jan-2035,uncategorized,low mileage unit. well maintained vehicle. vie...,factory radio setting. front recording camera....,NaN,43800.0
2,1026909,Mercedes-Benz CLA-Class CLA180,mercedes-benz,cla180,1 owner c&c unit. full agent service with 1 mo...,2016.0,NaN,25-jul-2016,luxury sedan,"parf car, premium ad car",...,80000.0,27886.0,26041.0,NaN,NaN,uncategorized,responsive and fuel efficient 1.6l inline 4 cy...,dual electric/memory seats. factory fitted aud...,NaN,95500.0
3,1019371,Mercedes-Benz E-Class E180 Avantgarde,mercedes-benz,e180,"fully agent maintained, 3 years warranty 10 ye...",2019.0,NaN,17-nov-2020,luxury sedan,"parf car, almost new car, consignment car",...,9800.0,46412.0,56977.0,NaN,NaN,uncategorized,"1.5l inline-4 twin scroll turbocharged engine,...",64 colour ambient lighting. active parking ass...,NaN,197900.0
4,1031014,Honda Civic 1.6A VTi,NaN,civic,"kah motor unit! 1 owner, lowest 1.98% for full...",2019.0,NaN,20-sep-2019,mid-sized sedan,parf car,...,40000.0,20072.0,20101.0,NaN,NaN,uncategorized,"1.6l i-vtec engine, 123 bhp, earth dreams cvt ...","s/rims, premium leather seats, factory touch s...",NaN,103200.0


# Data Preporcessing

In [115]:
def data_preprocess(df:pd.DataFrame) -> pd.DataFrame:
    df['make'] = df.apply(lambda row: row['title'].split()[0].lower() if pd.isna(row['make']) else row['make'],axis=1)
    df['make'] = utils.ordinal_encoder(df['make'])

    df['type_of_vehicle'] = utils.ordinal_encoder(df['type_of_vehicle'])
    df['transmission'] = utils.ordinal_encoder(df['transmission'])

    utils.fill_with_mean(df['power'])
    df['power'] = (df['power'] - np.min(df['power'])) / (np.max(df['power']) - np.min(df['power']))
    
    utils.fill_with_mean(df['engine_cap'])
    #df["engine_cap"] = utils.data_discretization(df["engine_cap"], num=10)
    df['engine_cap'] = (df['engine_cap'] - np.min(df['engine_cap'])) / (np.max(df['engine_cap']) - np.min(df['engine_cap']))
    

    df["depreciation"] = utils.del_outlier(df["depreciation"], lower_val=0.0, upper_val=0.99)
    utils.fill_with_mean(df['depreciation'])
    df['depreciation'] = (df['depreciation'] - np.min(df['depreciation'])) / (np.max(df['depreciation']) - np.min(df['depreciation']))

    utils.fill_with_mean(df["road_tax"])
    #df["road_tax"] = utils.data_discretization(df["road_tax"], num=15)
    df['road_tax'] = (df['road_tax'] - np.min(df['road_tax'])) / (np.max(df['road_tax']) - np.min(df['road_tax']))

    utils.fill_with_mean(df["mileage"])
    df["mileage"] = utils.data_discretization(df["mileage"], num=30)
    
    #df["depreciation"] = utils.data_discretization(df["depreciation"], num=15)
    #df["power"] = utils.data_discretization(df["power"], num=15)
    df["price"] = utils.data_discretization(df["price"], num=70)

        
    df.drop(columns='title', inplace=True)
    df.drop(columns='model', inplace=True)
    df.drop(columns='description', inplace=True)
    df.drop(columns='manufactured', inplace=True)
    df.drop(columns='original_reg_date', inplace=True)
    df.drop(columns='reg_date', inplace=True)
    df.drop(columns='fuel_type', inplace=True)
    df.drop(columns='opc_scheme', inplace=True)
    df.drop(columns='lifespan', inplace=True)
    df.drop(columns='eco_category', inplace=True)
    df.drop(columns='features', inplace=True)
    df.drop(columns='accessories', inplace=True)
    df.drop(columns='indicative_price', inplace=True)
    
    df.drop(columns='curb_weight', inplace=True)
    df.drop(columns='no_of_owners', inplace=True)
    df.drop(columns='coe', inplace=True)
    df.drop(columns='omv', inplace=True)
    df.drop(columns='category', inplace=True)
    df.drop(columns='arf', inplace=True)
    df.drop(columns='dereg_value', inplace=True)

    return df

In [116]:
df_process = data_preprocess(df_sample)

In [117]:
df_process

,listing_id,make,type_of_vehicle,transmission,power,engine_cap,depreciation,road_tax,mileage,price
0,1030324,1,1,1,0.184751,0.127352,0.191730,0.094118,3,2
1,1021510,2,2,2,0.182796,0.190166,0.111022,0.106207,5,2
2,1026909,3,1,1,0.096774,0.101715,0.156761,0.055984,4,3
3,1019371,3,1,1,0.145650,0.095466,0.174445,0.051440,1,5
4,1031014,4,3,1,0.100684,0.101843,0.095333,0.056146,2,3
...,...,...,...,...,...,...,...,...,...,...
16779,1030181,5,6,1,0.286413,0.125566,0.245180,0.091359,3,4
16780,1027041,7,5,1,0.123363,0.100759,0.099588,0.055335,5,2
16781,1021099,14,4,1,0.092864,0.101907,0.051988,0.062150,3,2
16782,1019473,4,5,1,0.063539,0.083987,0.061960,0.042840,1,2


## Computing the Top Recommendations

The method `get_top_recommendations()` shows an example of how to get the top recommendations for a given data sample (data sample = row in the dataframe of the dataset). The input is a row from the dataset and a list of optional input parameters which will depend on your approach; `k` is the number of returned recommendations seems useful, though.

The output should be a `pd.DataFrame` containing the recommendations. The output dataframe should have the same columns as the row + any additional columns you deem important (e.g., any score or tags that you might want to add to your recommendations).

In principle, the method `get_top_recommendations()` may be imported from a external Python (.py) script as well.

In [118]:
def get_top_recommendations(row, **kwargs) -> pd.DataFrame:
    
    #####################################################
    ## Initialize the required parameters
    
    # The number of recommendations seem recommended
    # Additional input parameters are up to you
    k = None
    source_data = None
    mean = 'cos'
    
    # Extract all **kwargs input parameters
    # and set the used paramaters (here: k)
    for key, value in kwargs.items():
        if key == 'k':
            k = value
        if key == 'source_data':
            source_data = value
        if key == 'mean':
            mean = value
            
       
    #####################################################
    ## Compute your recommendations
    #
    # This is where your magic happens. Of course, you can call methods
    # defined in this notebook or in external Python (.py) scripts

    # slice
    column = ["price","depreciation","road_tax","make","power","engine_cap","mileage","type_of_vehicle"]
    selected_source_data = source_data[column].copy()  # reassign
    selected_row = row[column]

    # convert to 2-d numpy.array
    np_source_data = np.array(selected_source_data.values, dtype=np.float32)
    np_row = np.array(selected_row.values, dtype=np.float32).reshape(1,-1)
    
    ## Different similarity metric functions
    top_k_index = None
    row_index = row.name # exclude row to avoid recommending the item itself
    
    if mean == 'dis':
        # weighted
        weights = np.array([5,2,1,2,1,1,1,1], dtype=np.float32)
        # calculate weighted distance and reshape to 1-d
        num = (np_source_data - np_row)*weights
        result = np.linalg.norm(num,axis=1,keepdims=True).reshape(-1)
        # exclude the row itself
        result[row_index] = np.inf
        # indices of top k most similar items
        top_k_index = heapq.nsmallest(k, range(len(result)), result.take)
    else:
        # calculate cos simularity and reshape to 1-d
        num = np.dot(np_row,np_source_data.T).reshape(-1)
        denom = np.linalg.norm(np_row,axis=1,keepdims=True).reshape(-1)*np.linalg.norm(np_source_data,axis=1,keepdims=True).reshape(-1)
        result = np.round(num/denom, 2)
        # exclude the row itself
        result[row_index] = -1
        # indices of top k most similar items
        top_k_index = heapq.nlargest(k, range(len(result)), result.take)
    
    print(top_k_index)
    
    
    ##################################################### 
    ## Return
    # Here, we just return the input row k times
    # Ideally, you recommendations will be much better
    df_result = pd.DataFrame(source_data.loc[top_k_index], index=None)
        
    # Return the dataset with the k recommendations
    return df_result

## Testing the Recommendation Engine

This will be the main part of your notebook to allow for testing your solutions. Most basically, for a given listing (defined by the row id in your input dataframe), we would like to see the recommendations you make. So however you set up your notebook, it should have at least a comparable section that will allow us to run your solution for different inputs.

### Pick a Sample Listing as Input

In [120]:
# Pick a row id of choice
row_id = 10
#row_id = 20
#row_id = 30
#row_id = 40
#row_id = 50

# Get the row from the dataframe (an valid row ids will throw an error)
row = df_process.iloc[row_id]

# Just for printing it nicely, we create a new dataframe from this single row
pd.DataFrame([row])

,listing_id,make,type_of_vehicle,transmission,power,engine_cap,depreciation,road_tax,mileage,price
10,1004029,6,3,1,0.107136,0.10146,0.066348,0.055822,3,2


## Compute and Display the recommendations

Since the method `get_top_recommendations()` returns a `pd.DataFrame`, it's easy to display the result.

In [121]:
k = 10

df_recommendations = get_top_recommendations(row, k=k, source_data=df_sample, mean='dis')

df_recommendations.head(k)

[895, 8599, 7653, 6388, 7802, 513, 3800, 405, 9655, 6670]


,listing_id,make,type_of_vehicle,transmission,power,engine_cap,depreciation,road_tax,mileage,price
895,1029390,6,3,1,0.107136,0.10146,0.065683,0.055822,3,2
8599,1004269,6,3,1,0.107136,0.10146,0.067145,0.055822,3,2
7653,992386,6,3,1,0.107136,0.10146,0.065284,0.055822,3,2
6388,1016745,6,3,1,0.107136,0.10146,0.067544,0.055822,3,2
7802,1023921,6,3,1,0.107136,0.10146,0.067544,0.055822,3,2
513,1008660,6,3,1,0.107136,0.10146,0.067810,0.055822,3,2
3800,1011395,6,3,1,0.107136,0.10146,0.067810,0.055822,3,2
405,1019476,6,3,1,0.107136,0.10146,0.064752,0.055822,3,2
9655,1006884,6,3,1,0.107136,0.10146,0.064353,0.055822,3,2
6670,1022268,6,3,1,0.104203,0.10146,0.067943,0.055822,3,2


In [122]:
recommendation = df_recommendations.head(k)
df_recommendation = pd.DataFrame()
for i in recommendation['listing_id']:
    df_recommendation = df_recommendation.append(df_original[df_original['listing_id'] == i])

In [123]:
df_recommendation

,listing_id,title,make,model,description,manufactured,original_reg_date,reg_date,type_of_vehicle,category,...,mileage,omv,arf,opc_scheme,lifespan,eco_category,features,accessories,indicative_price,price
895,1029390,Kia Cerato K3 1.6A,kia,cerato,"immaculate condition, well taken care of by me...",2017.0,NaN,21-apr-2017,mid-sized sedan,parf car,...,50000.0,13048.0,13048.0,NaN,NaN,uncategorized,"6 speed auto transmission, ex model, toggle be...",d-cut carbon fiber steering with multi-control...,NaN,57800.0
8599,1004269,Kia Cerato K3 1.6A,NaN,cerato,"fully serviced and maintained by c&c, 5 years ...",2017.0,NaN,03-jan-2018,mid-sized sedan,parf car,...,62000.0,11650.0,11650.0,NaN,NaN,uncategorized,"1.6l 4 cylinders, dual cvvt engine, responsive...",factory fitted infotainment system with revers...,NaN,64700.0
7653,992386,Kia Cerato K3 1.6A,NaN,cerato,"1.98% interest, 2 years extended warranty , fu...",2017.0,NaN,22-jan-2018,mid-sized sedan,parf car,...,51240.0,12547.0,12547.0,NaN,NaN,uncategorized,"1.6l 4 cylinders, dual cvvt engine, 6-speed au...","leather seat, sport rim, factory fitted infota...",NaN,64700.0
6388,1016745,Kia Cerato K3 1.6A,kia,cerato,2,2017.0,NaN,15-aug-2017,mid-sized sedan,"parf car, premium ad car",...,65803.0,11681.0,11681.0,NaN,NaN,uncategorized,4 cylinder inline 16 valve dohc dual cvvt engi...,"sports rims, leather seats, factory audio syst...",NaN,61400.0
7802,1023921,Kia Cerato K3 1.6A Sunroof,NaN,cerato,"superbly well maintained by sole owner, afford...",2017.0,NaN,27-sep-2017,mid-sized sedan,"parf car, premium ad car",...,NaN,15768.0,15768.0,NaN,NaN,uncategorized,1.6l four cylinder in-line 16 valve dohc dual ...,"original bodykit, sunroof, ventilated electric...",NaN,64700.0
513,1008660,Kia Cerato K3 1.6A,kia,cerato,interest rate 1.88%. 1 owner only. car in pris...,2017.0,NaN,30-mar-2017,mid-sized sedan,"parf car, premium ad car",...,NaN,12181.0,12181.0,NaN,NaN,uncategorized,"1.6l dual cvvt engine. 127 bhp, 6 speed auto w...","premium leather seats, factory audio system bu...",NaN,58300.0
3800,1011395,Kia Cerato K3 1.6A EX,kia,cerato,"1 owner unit, ex model, sporty sedan, maintain...",2017.0,NaN,22-aug-2017,mid-sized sedan,parf car,...,73000.0,12918.0,12918.0,NaN,NaN,uncategorized,"1.6l, 4 cylinders inline 16 valve dohc dual cv...","leather seats, sports rims, factory audio syst...",NaN,62400.0
405,1019476,Kia Cerato K3 1.6A,NaN,cerato,"viewing by appt only, whatsapp for doorstep vi...",2017.0,NaN,18-sep-2017,mid-sized sedan,parf car,...,NaN,12763.0,12763.0,NaN,NaN,uncategorized,powerful and fuel efficient 1.6l dohc dual cvv...,sports rims. audio system aux and usb port. fo...,NaN,61400.0
9655,1006884,Kia Cerato K3 1.6A,NaN,cerato,623,2017.0,NaN,20-jul-2017,mid-sized sedan,parf car,...,NaN,11884.0,11884.0,NaN,NaN,uncategorized,"1.6l 4 cylinders 16 valves, dohc dual cvvt eng...",multi-function steering wheel. leather seats. ...,NaN,59300.0
6670,1022268,Kia Cerato 1.6A EX,kia,cerato,1 owner only. promo 1.88% bank loan interest r...,2018.0,NaN,06-may-2019,mid-sized sedan,parf car,...,50000.0,14385.0,14385.0,NaN,NaN,uncategorized,"1.6l engine with dual-cvvt technology, 125bhp/...","sports rims, keyless entry, usb charging port,...",NaN,79000.0
